In [1]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
import logging

## Loading data

In [2]:
DATA_PATH = '../data/raw/'

subset_path = os.path.join(DATA_PATH, 'subset')
export_path = '../data/processed/subset_input'
os.makedirs(export_path, exist_ok=True)

name_map = {
    'AR drone': 'ar',
    'Bepop drone': 'bepop',
    'Background RF activities': 'bg',
    'Phantom drone': 'phantom'
}

sample_rate = 40e6
nperseg = 1024

## Retreiving power levels for each frequency bin

In [71]:
from scipy.fft import fft
from scipy.signal import welch
def get_power_levels(file_path, sample_rate=40e6, nperseg=1024):
    """
    Computes the FFT for the segment. The number of unique frequency bins is (nperseg / 2) + 1 = 513.
    The extra '+1' is due the DC component and the Nyquist frequency. 

    Returns a data frame with 513 frequency bins ranging from 0 Hz to 20MHz (half of sampling rate) 
    and the power level for each frequency bin.
    """
    data = pd.read_csv(file_path, header=None).values.flatten()
    
    frequencies, psd = welch(data, fs=sample_rate, nperseg=nperseg, scaling='density')
    
    # Convert to dB
    psd_db = 10 * np.log10(psd)
    
    df = pd.DataFrame({'bin_freq': frequencies, 'db': psd_db})
    
    return df

In [72]:
def process_all_segments(baseline_path, sample_rate, nperseg, num_segments=5):
    """
    For a specified number of segments, a combined data frame for the power levels for all segments is returned.
    """
    all_power_levels = []
    counter = 0
    for filename in os.listdir(baseline_path):
        if counter == num_segments:
            break
        if filename.endswith('.csv'):
            file_path = os.path.join(baseline_path, filename)
            print(file_path)
            power_levels = get_power_levels(file_path, sample_rate, nperseg)
            all_power_levels.append(power_levels)
            counter += 1
    return pd.concat(all_power_levels, ignore_index=True)

Run this line ONLY when you need data.

In [80]:
baseline_path = os.path.join(subset_path, 'bg')

In [8]:
all_segment_data = process_all_segments(baseline_path, sample_rate, nperseg)

../data/raw/subset/bg/00000L_28.csv
../data/raw/subset/bg/00000L_14.csv
../data/raw/subset/bg/00000L_15.csv
../data/raw/subset/bg/00000L_29.csv
../data/raw/subset/bg/00000L_17.csv
../data/raw/subset/bg/00000L_16.csv


## Calculate baseline stats and detect anomalies through sigmoid likelihoods

In [81]:
def calculate_baseline_stats(all_segment_data):
    """
    Groups the baseline data by frequency bin and calculates mean and standard deviation for each frequency bin.
    """
    stats = all_segment_data.groupby(by=['bin_freq']).agg({'db': ['mean', 'std']})
    return stats

In [93]:
def detect_anomalies_z_score(test_power_levels, stats, threshold=0.5):
    """
    Merges test sample power levels with baseline stats and returns the merged data frame 
    and and array of anomalies after comparing z-scores with threshold
    """
    mean_df = stats['db']['mean'].reset_index()
    stds_df = stats['db']['std'].reset_index()
    merged_df = pd.merge(pd.merge(test_power_levels, mean_df, on='bin_freq'), stds_df, on='bin_freq')

    merged_df['z_score'] = (merged_df['db'] - merged_df['mean']) / (merged_df['std'])
    merged_df['probs'] = 1 / (1 + np.exp(-(2 * merged_df['z_score'] - 3)))
    
    anomalies = merged_df['probs'] > threshold
    
    return merged_df, anomalies

In [94]:
def print_if_anomaly(anomalies):
    anomaly_prop = (anomalies.sum() / len(anomalies)) * 100
    if anomaly_prop > 50:
        print(f"Anomaly detected! {anomaly_prop:.2f}% of frequency bins are anomalous.")
    else:
        print(f"Normal signal detected! {anomaly_prop:.2f}% of frequency bins are anomalous.")

In [95]:
stats = calculate_baseline_stats(all_segment_data)

In [96]:
print(stats)

                   db          
                 mean       std
bin_freq                       
0.0        -71.945662  0.041916
39062.5    -64.973211  0.031848
78125.0    -64.180423  0.034899
117187.5   -64.175892  0.030306
156250.0   -64.176783  0.023045
...               ...       ...
19843750.0 -64.168613  0.021449
19882812.5 -64.178771  0.043482
19921875.0 -64.162850  0.033870
19960937.5 -63.535991  0.039484
20000000.0 -65.104129  0.069622

[513 rows x 2 columns]


In [97]:
print(stats.columns)

MultiIndex([('db', 'mean'),
            ('db',  'std')],
           )


In [98]:
print(stats.reset_index().columns)

MultiIndex([('bin_freq',     ''),
            (      'db', 'mean'),
            (      'db',  'std')],
           )


In [100]:
drone_path = os.path.join(subset_path, 'drone/10100L_2.csv')
drone_power_levels = get_power_levels(drone_path)

In [101]:
bg_path = os.path.join(subset_path, 'bg/00000L_2.csv')
bg_power_levels = get_power_levels(bg_path)

In [102]:
merged_drone_df, drone_anomalies = detect_anomalies_z_score(drone_power_levels, stats, threshold=0.5)

In [103]:
_, bg_anomalies = detect_anomalies_z_score(bg_power_levels, stats, threshold=0.5)

In [104]:
print_if_anomaly(drone_anomalies)
print_if_anomaly(bg_anomalies)

Anomaly detected! 99.81% of frequency bins are anomalous.
Normal signal detected! 9.75% of frequency bins are anomalous.


## Isolation Forest

In [111]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

In [108]:
X = all_segment_data.drop('bin_freq', axis=1)

In [109]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [112]:
iso_forest = IsolationForest(random_state=42)
iso_forest.fit(X_scaled)

IsolationForest(random_state=42)

In [114]:
anomalies = iso_forest.predict(X_scaled)

# Add anomaly predictions to the dataset
all_segment_data['anomaly'] = anomalies

In [119]:
print(anomalies)

[-1  1  1 ...  1  1  1]


In [ ]:
prediction

In [130]:
def detect_anomalies_iso_forest(test_data, threshold=50):
    test_data_scaled = scaler.transform(test_data.drop('bin_freq', axis=1))
    prediction = iso_forest.predict(test_data_scaled)
    anomaly_count = (prediction == -1).sum()
    total_bins = len(prediction)
    anomaly_percentage = (anomaly_count / total_bins) * 100
    print(f"Number of anomalies: {anomaly_count}")
    print(f"Percentage of anomalies: {anomaly_percentage:.2f}%")
    return "Anomaly Detected" if anomaly_percentage > threshold else "Normal"

In [133]:
detect_anomalies_iso_forest(drone_power_levels)

Number of anomalies: 512
Percentage of anomalies: 99.81%


'Anomaly Detected'

In [134]:
detect_anomalies_iso_forest(bg_power_levels)

Number of anomalies: 181
Percentage of anomalies: 35.28%


'Normal'